In [1]:
#Importation of Libraries
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split  # Ajoutez cette ligne
import xgboost as xgb
from sklearn.metrics import accuracy_score
import nevergrad as ng

In [2]:
df = pd.read_csv('x_train_preprocessed.csv', header=0)
x_train_preprocessed = df.copy()
x_train_preprocessed= np.array(x_train_preprocessed)

In [3]:
y_train = pd.read_csv('y_train.csv', index_col=['id'])
labels_raw = y_train.copy() # a copy of the raw labels
labels_sorted = y_train.copy().sort_values(by = ['y']) # a copy of the raw labels sorted
#labels_raw.head()

In [4]:
data = np.array(df)
l = np.array(y_train)
labels = [l[i][0] for i in range(len(l))]


In [5]:
%pip install nevergrad


Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install numpy

Note: you may need to restart the kernel to use updated packages.


In [5]:
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nevergrad as ng
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

def train_evaluate(hyperparameters):
    
    model = CatBoostClassifier(
        iterations=int(hyperparameters['iterations']),
        learning_rate=hyperparameters['learning_rate'],
        random_seed=int(hyperparameters['random_seed']),
        l2_leaf_reg=hyperparameters['l2_leaf_reg'],
        bagging_temperature = hyperparameters['bagging_temperature']
        
        #n_estimators=int(hyperparameters['n_estimators']),
        #max_depth=int(hyperparameters['max_depth']),
        
        #subsample=hyperparameters['subsample'],
        #colsample_bytree=hyperparameters['colsample_bytree']
    )

    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    return -accuracy_score(y_test, preds)  

parametrization = ng.p.Dict(
    iterations = ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
    learning_rate = ng.p.Log(lower=0.01, upper=1),
    random_seed = ng.p.Scalar(lower=0, upper=1000).set_integer_casting(),
    l2_leaf_reg = ng.p.Scalar(lower=0, upper=1000),
    bagging_temperature = ng.p.Scalar(lower=0, upper=1000)
    #n_estimators=ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
    #max_depth=ng.p.Scalar(lower=3, upper=10).set_integer_casting(),
    #learning_rate=ng.p.Log(lower=0.01, upper=1),
    #subsample=ng.p.Scalar(lower=0.5, upper=1),
    #colsample_bytree=ng.p.Scalar(lower=0.5, upper=1)
)


optimizer = ng.optimizers.TwoPointsDE(parametrization=parametrization, budget=100)

for _ in range(optimizer.budget):
    x = optimizer.ask()
    loss = train_evaluate(x.value)
    optimizer.tell(x, loss)

# Meilleurs hyperparamètres trouvés
best_hyperparams = optimizer.recommend().value
print("Catboost:", best_hyperparams)

final_model = CatBoostClassifier(**best_hyperparams)
final_model.fit(data, y_train)
final_predictions = final_model.predict(X_test)


0:	learn: 1.0871205	total: 77.9ms	remaining: 58.6s
1:	learn: 1.0254027	total: 80.4ms	remaining: 30.2s
2:	learn: 1.0034691	total: 82.9ms	remaining: 20.7s
3:	learn: 0.9950432	total: 85.4ms	remaining: 16s
4:	learn: 0.9916092	total: 87.9ms	remaining: 13.2s
5:	learn: 0.9901216	total: 90.1ms	remaining: 11.2s
6:	learn: 0.9894391	total: 92.3ms	remaining: 9.85s
7:	learn: 0.9891109	total: 94.7ms	remaining: 8.83s
8:	learn: 0.9889475	total: 97.2ms	remaining: 8.05s
9:	learn: 0.9888641	total: 99.3ms	remaining: 7.39s
10:	learn: 0.9888209	total: 102ms	remaining: 6.86s
11:	learn: 0.9887982	total: 104ms	remaining: 6.42s
12:	learn: 0.9887863	total: 106ms	remaining: 6.06s
13:	learn: 0.9887799	total: 108ms	remaining: 5.73s
14:	learn: 0.9887766	total: 110ms	remaining: 5.44s
15:	learn: 0.9887748	total: 113ms	remaining: 5.21s
16:	learn: 0.9887738	total: 116ms	remaining: 5.01s
17:	learn: 0.9887733	total: 118ms	remaining: 4.81s
18:	learn: 0.9887730	total: 120ms	remaining: 4.64s
19:	learn: 0.9887729	total: 122ms

CatBoostError: Length of label=4093 and length of data=5117 is different.

In [6]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import nevergrad as ng
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

def train_evaluate(hyperparameters):
    
    model = XGBClassifier(
        n_estimators=int(hyperparameters['n_estimators']),
        max_depth=int(hyperparameters['max_depth']),
        learning_rate=hyperparameters['learning_rate'],
        subsample=hyperparameters['subsample'],
        colsample_bytree=hyperparameters['colsample_bytree']
    )

    model.fit(X_train,y_train)
    preds = model.predict(X_test)
    return -accuracy_score(y_test, preds)  

parametrization = ng.p.Dict(
    n_estimators=ng.p.Scalar(lower=10, upper=1000).set_integer_casting(),
    max_depth=ng.p.Scalar(lower=3, upper=10).set_integer_casting(),
    learning_rate=ng.p.Log(lower=0.01, upper=1),
    subsample=ng.p.Scalar(lower=0.5, upper=1),
    colsample_bytree=ng.p.Scalar(lower=0.5, upper=1)
)


optimizer = ng.optimizers.TwoPointsDE(parametrization=parametrization, budget=100)

for _ in range(optimizer.budget):
    x = optimizer.ask()
    loss = train_evaluate(x.value)
    optimizer.tell(x, loss)

# Meilleurs hyperparamètres trouvés
best_hyperparams = optimizer.recommend().value
print("Meilleurs hyperparamètres trouvés:", best_hyperparams)

final_model = XGBClassifier(**best_hyperparams)
final_model.fit(data, y_train)
final_predictions = final_model.predict(X_test)


Meilleurs hyperparamètres trouvés: {'n_estimators': 394, 'max_depth': 4, 'learning_rate': 0.10577963049602537, 'subsample': 0.8395886276456808, 'colsample_bytree': 0.7447914488185501}


XGBoostError: [18:30:27] /workspace/src/data/data.cc:501: Check failed: this->labels.Size() % this->num_row_ == 0 (4093 vs. 0) : Incorrect size for labels.
Stack trace:
  [bt] (0) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x3581ea) [0x7f74275091ea]
  [bt] (1) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x389457) [0x7f742753a457]
  [bt] (2) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(+0x38a4b1) [0x7f742753b4b1]
  [bt] (3) /home/linuxbrew/.linuxbrew/Cellar/python@3.11/3.11.5/lib/python3.11/site-packages/xgboost/lib/libxgboost.so(XGDMatrixSetInfoFromInterface+0xb0) [0x7f742730f210]
  [bt] (4) /home/linuxbrew/.linuxbrew/opt/libffi/lib/libffi.so.8(+0x9056) [0x7f74aa284056]
  [bt] (5) /home/linuxbrew/.linuxbrew/opt/libffi/lib/libffi.so.8(+0x7ae9) [0x7f74aa282ae9]
  [bt] (6) /home/linuxbrew/.linuxbre